# Credit Card Default Prediction

## Data Preprocessing

In [ ]:
from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier, plot_importance
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import KMeansSMOTE
import shap

In [ ]:
# Avoid warnings caused by inplace column name reassignment
pd.options.mode.chained_assignment = None

# fetch dataset 
default_of_credit_card_clients = fetch_ucirepo(id=350) 
  
# data (as pandas dataframes) 
features = default_of_credit_card_clients.data.features
targets = default_of_credit_card_clients.data.targets

Rename feature columns to be more descriptive

In [ ]:
# Rename columns for better understanding
features.rename(inplace=True,
                columns={
                    'X1': 'CREDIT_LIMIT',  # Credit limit (NT dollar)
                    'X2': 'GENDER',  # Gender (1 = male; 2 = female)
                    'X3': 'EDUCATION_LEVEL',  # Education (1 = graduate school; 2 = university; 3 = high school; 4 = others)
                    'X4': 'MARITAL_STATUS',  # Marital status (1 = married; 2 = single; 3 = others)
                    'X5': 'AGE',  # (years)

                    # X6 - X11 is repayment status
                    # The measurement scale for the repayment status is:
                    # -1 = pay duly;
                    # 1 = payment delay for one month;
                    # 2 = payment delay for two months;
                    # . . .;
                    # 8 = payment delay for eight months;
                    # 9 = payment delay for nine months and above.
                    'X6': 'SEPT_PAY_STATUS',  # repayment status in September, 2005
                    'X7': 'AUG_PAY_STATUS',  # repayment status in August, 2005
                    'X8': 'JULY_PAY_STATUS',  # repayment status in July, 2005
                    'X9': 'JUNE_PAY_STATUS',  # repayment status in June, 2005
                    'X10': 'MAY_PAY_STATUS',  # repayment status in May, 2005
                    'X11': 'APRIL_PAY_STATUS',  # repayment status in April, 2005

                    # X12 - X17 is amount of bill statement (NT dollar)
                    'X12': 'SEPT_BILL',  # amount of bill statement in September, 2005
                    'X13': 'AUG_BILL',  # amount of bill statement in August, 2005
                    'X14': 'JULY_BILL',  # amount of bill statement in July, 2005
                    'X15': 'JUNE_BILL',  # amount of bill statement in June, 2005
                    'X16': 'MAY_BILL',  # amount of bill statement in May, 2005
                    'X17': 'APRIL_BILL',  # amount of bill statement in April, 2005

                    # X18 - X23 is amount of previous payment (NT dollar)
                    'X18': 'SEPT_PAYMENT',  # amount paid in September, 2005
                    'X19': 'AUG_PAYMENT',  # amount paid in August, 2005
                    'X20': 'JULY_PAYMENT',  # amount paid in July, 2005
                    'X21': 'JUNE_PAYMENT',  # amount paid in June, 2005
                    'X22': 'MAY_PAYMENT',  # amount paid in May, 2005
                    'X23': 'APRIL_PAYMENT',  # amount paid in April, 2005
                })

# Display the updated dataframe
features.head()


Rename target column to be more descriptive

In [ ]:
targets.rename(inplace = True,
               columns={'Y': 'DEFAULT'} # Default payment next month
              )

targets.head()

## Exploratory Data Analysis

In [ ]:
# Combine features and targets for EDA
data = pd.concat([features, targets], axis=1)

# Check for missing values
print(data.isnull().sum())

# Summary statistics
print(data.describe())

# Plot distributions of numerical variables
numerical_features = [
    'CREDIT_LIMIT', 'AGE', 'SEPT_BILL', 'AUG_BILL', 'JULY_BILL',
    'JUNE_BILL', 'MAY_BILL', 'APRIL_BILL', 'SEPT_PAYMENT',
    'AUG_PAYMENT', 'JULY_PAYMENT', 'JUNE_PAYMENT', 'MAY_PAYMENT',
    'APRIL_PAYMENT'
]

plt.figure(figsize=(12, 8))
data[numerical_features].hist(bins=20, figsize=(20, 15))
plt.tight_layout()
plt.show()



## Outlier Detection and Handling

In [ ]:
def cap_outliers(data, columns):
    for column in columns:
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)][column]
        print(f"{column}: {len(outliers)} outliers capped.")
        data[column] = data[column].clip(lower=lower_bound, upper=upper_bound)
    return data

features = cap_outliers(features, numerical_features)



## Feature Engineering

In [ ]:
# Encode categorical variables
features['GENDER'] = features['GENDER'].map({1: 0, 2: 1})  # 1: Male -> 0, 2: Female -> 1
print("GENDER column transformed:", features['GENDER'].unique())

features['EDUCATION_LEVEL'] = features['EDUCATION_LEVEL'].replace({0: 4, 5: 4, 6: 4})  # 4: Others
print("EDUCATION_LEVEL column transformed:", features['EDUCATION_LEVEL'].unique())

features['MARITAL_STATUS'] = features['MARITAL_STATUS'].replace({0: 3})  # 3: Others
print("MARITAL_STATUS column transformed:", features['MARITAL_STATUS'].unique())

# Combine processed features with targets
processed_data = pd.concat([features, targets], axis=1)
print("Processed data sample:")
print(processed_data.head())

## Normalized Data After Outlier Handling

In [ ]:
# Normalize numerical columns using MinMaxScaler
scaler = MinMaxScaler()
features[numerical_features] = scaler.fit_transform(features[numerical_features])

# Combine normalized features with targets for inspection
normalized_data = pd.concat([features, targets], axis=1)

# Display normalized data sample
print("Normalized data sample:")
print(normalized_data.head())

# Check the initial class distribution
print("\nOriginal class distribution:")
print(targets.value_counts())


## Set Up K-Fold Cross Validation

In [ ]:
# Step 1: Create a test set
x_train_val, x_test, y_train_val, y_test = train_test_split(
    features, targets['DEFAULT'], test_size=0.2, random_state=42, stratify=targets['DEFAULT']
)

# Step 2: Perform K-Fold Cross-Validation on the remaining train+validation set
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=0)

train_indices = []
validation_indices = []

for fold, (train_idx, val_idx) in enumerate(kf.split(x_train_val)):
    train_indices.append(train_idx)
    validation_indices.append(val_idx)
    print(f"Fold {fold + 1}:")
    print(f"  Training indices: {train_idx[:5]}... ({len(train_idx)} samples)")
    print(f"  Validation indices: {val_idx[:5]}... ({len(val_idx)} samples)")

## Apply K-Means SMOTE and Train the Model in Each Fold

In [ ]:
# Step 1: Create train/validation and test sets
x_train_val, x_test, y_train_val, y_test = train_test_split(
    features, targets['DEFAULT'], test_size=0.2, random_state=42, stratify=targets['DEFAULT']
)

# Step 2: Initialize KMeansSMOTE and GaussianNB
kmeans_smote = KMeansSMOTE(random_state=0, k_neighbors=5, cluster_balance_threshold=0.1)
model = GaussianNB()  # Replace with your preferred model

# Step 3: Perform K-Fold Cross-Validation for train/validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=0)

# Store scores for each fold
train_scores = []
validation_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(x_train_val)):
    print(f"\nFold {fold + 1}/{n_splits}")
    
    # Get training and validation data for the current fold
    x_fold_train, x_fold_val = x_train_val.iloc[train_idx], x_train_val.iloc[val_idx]
    y_fold_train, y_fold_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]
    
    # Apply KMeansSMOTE to the training data
    x_fold_train_resampled, y_fold_train_resampled = kmeans_smote.fit_resample(x_fold_train, y_fold_train)
    
    # Train the model on the resampled training data
    model.fit(x_fold_train_resampled, y_fold_train_resampled)
    
    # Evaluate on the training set
    y_train_pred = model.predict(x_fold_train_resampled)
    train_accuracy = accuracy_score(y_fold_train_resampled, y_train_pred)
    train_scores.append(train_accuracy)
    
    # Evaluate on the validation set
    y_val_pred = model.predict(x_fold_val)
    val_accuracy = accuracy_score(y_fold_val, y_val_pred)
    validation_scores.append(val_accuracy)
    
    print(f"  Training Accuracy: {train_accuracy:.2f}")
    print(f"  Validation Accuracy: {val_accuracy:.2f}")

# Step 4: Calculate average scores across all folds
average_train_score = sum(train_scores) / n_splits
average_val_score = sum(validation_scores) / n_splits

print("\nK-Fold Cross-Validation with K-Means SMOTE Results:")
print(f"  Average Training Accuracy: {average_train_score:.2f}")
print(f"  Average Validation Accuracy: {average_val_score:.2f}")

# Step 5: Final evaluation on the test set
x_test_resampled, y_test_resampled = kmeans_smote.fit_resample(x_train_val, y_train_val)  # Fit SMOTE to the full train/val data
model.fit(x_test_resampled, y_test_resampled)  # Retrain on full train/val data

# Test set evaluation
y_test_pred = model.predict(x_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"\nTest Set Accuracy: {test_accuracy:.2f}")

# XGBoost

### Recursive Feature Elimination using Logistic Regression

In [ ]:
# Normalize numerical features (check if this is needed)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Initialize Logistic Regression model
logistic_model = LogisticRegression(max_iter=1000, random_state=42)

# Perform Recursive Feature Elimination (RFE)
rfe_logistic = RFE(estimator=logistic_model, n_features_to_select=1, step=1)
fit_logistic = rfe_logistic.fit(features_scaled, targets['DEFAULT'].values.ravel())

# Create a DataFrame for feature rankings
all_features_ranking = pd.DataFrame({
    'Feature': features.columns,
    'Ranking': fit_logistic.ranking_,
}).sort_values(by='Ranking')

# Display top-ranked features
print("Feature Ranking (Top 10):")
print(all_features_ranking.head(10))

### Baseline XGBoost with all features

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(features, targets, test_size=0.2, random_state=42)
xgb_model_all_features = XGBClassifier(eval_metric='logloss')
xgb_model_all_features.fit(features_train, target_train)

pred = xgb_model_all_features.predict(features_test)
score = accuracy_score(target_test, pred)
print('Accuracy score using all features: {}'.format(f"{score:.4f}"))
print("\nClassification Report:")
print(classification_report(target_test, pred))

plot_importance(xgb_model_all_features, importance_type='gain')
plt.title('Feature Importance by Gain')
plt.show()

## XGBoost with top 10 features by Gain

In [ ]:
feature_importance = xgb_model_all_features.get_booster().get_score(importance_type='gain')
feature_importance_df = pd.DataFrame.from_dict(feature_importance, orient='index', columns=['Gain']).sort_values(by='Gain', ascending=False)
top_10_features = feature_importance_df.head(10).index.tolist()

xgb_model_top_10_features = XGBClassifier(eval_metric='logloss')
xgb_model_top_10_features.fit(features_train[top_10_features], target_train)

pred = xgb_model_top_10_features.predict(features_test[top_10_features])
score = accuracy_score(target_test, pred)
print('Accuracy score using top 10 features: {}'.format(score))
print("\nClassification Report:")
print(classification_report(target_test, pred))

plot_importance(xgb_model_top_10_features, importance_type='gain')
plt.title('Feature Importance by Gain')
plt.show()

### XGBoost without demographic data

In [ ]:
features_train_no_demographics = features_train.drop(['GENDER', 'AGE', 'EDUCATION_LEVEL', 'MARITAL_STATUS'], axis=1)
features_test_no_demographics = features_test.drop(['GENDER', 'AGE', 'EDUCATION_LEVEL', 'MARITAL_STATUS'], axis=1)

xgb_model_no_demographics = XGBClassifier(eval_metric='logloss')
xgb_model_no_demographics.fit(features_train_no_demographics, target_train)

pred = xgb_model_no_demographics.predict(features_test_no_demographics)
score = accuracy_score(target_test, pred)
print('Accuracy score using all features: {}'.format(score))
print("\nClassification Report:")
print(classification_report(target_test, pred))

plot_importance(xgb_model_no_demographics, importance_type='gain')
plt.title('Feature Importance by Gain')
plt.show()

### Use only 3 most recent months in features 

In [ ]:
# Drop recent month-related bill and payment features
recent_features = ['APRIL_BILL', 'APRIL_PAYMENT', 'MAY_BILL', 'MAY_PAYMENT', 'JUNE_BILL', 'JUNE_PAYMENT']
features_train_no_demo_most_recent = features_train_no_demographics.drop(recent_features, axis=1)
features_test_no_demo_most_recent = features_test_no_demographics.drop(recent_features, axis=1)

# Train XGBoost model
xgb_model_no_demo_most_recent = XGBClassifier(eval_metric='logloss', random_state=42)
xgb_model_no_demo_most_recent.fit(features_train_no_demo_most_recent, target_train)

# Evaluate the model
pred = xgb_model_no_demo_most_recent.predict(features_test_no_demo_most_recent)
score = accuracy_score(target_test, pred)
print(f'Accuracy score without demographics and recent months: {score:.4f}')
print("\nClassification Report:")
print(classification_report(target_test, pred))

# Plot feature importance
plot_importance(xgb_model_no_demo_most_recent, importance_type='gain', max_num_features=10, height=0.5)
plt.title('Top 10 Feature Importance by Gain (Excluding Demographics and Recent Months)', fontsize=14)
plt.show()


# SHAP (SHapley Additive exPlanations)

### Feature Importance via SHAP Summary Plot

In [ ]:
#xgboost with all features 

# Create a SHAP explainer
explainer = shap.Explainer(xgb_model_all_features, x_fold_train_resampled)

# Compute SHAP values
shap_values = explainer(x_fold_train_resampled)

# SHAP summary plot
plt.figure(figsize=(12, 6))
shap.summary_plot(shap_values, x_fold_train_resampled)

In [ ]:
#xgboost with no demographics 

# Create a SHAP explainer
explainer = shap.Explainer(xgb_model_no_demographics, x_fold_train_resampled)

# Compute SHAP values
shap_values = explainer(x_fold_train_resampled)

# SHAP summary plot
plt.figure(figsize=(12, 6))
shap.summary_plot(shap_values, x_fold_train_resampled)

### Feature Contribution for a Single Prediction


In [ ]:
x_test_scaled_df = pd.DataFrame(x_fold_train_resampled, columns=features.columns)

# Choose a specific instance from the test set
## Why couldnt we pick aggregate (all rows) instead of indexing at 0 and only picking 1 row? 
instance_index = 0  # Adjust the index as needed
instance_data = x_test_scaled_df.iloc[instance_index:instance_index+1] #+1 is to extract row as dataframe instead of a series 

# Generate SHAP values for the instance
shap_values_instance = explainer.shap_values(instance_data)

# SHAP waterfall plot
plt.figure(figsize=(12, 6))
shap.waterfall_plot(
    shap.Explanation(
        values=shap_values_instance[0],
        base_values=explainer.expected_value,
        data=instance_data.iloc[0],
        feature_names=features.columns  # Ensures feature names are shown
    )
)


In [ ]:
# Visualizing the normalized data

# 1. Boxplots of normalized numerical features
plt.figure(figsize=(15, 8))
sns.boxplot(data=features[numerical_features])
plt.title("Boxplots of Normalized Numerical Features")
plt.xticks(rotation=45)
plt.show()

# 2. Histograms of normalized features
features[numerical_features].hist(bins=20, figsize=(15, 10))
plt.suptitle("Histograms of Normalized Numerical Features", fontsize=16)
plt.tight_layout()
plt.show()



In [ ]:
# Compute and visualize the correlation matrix
normalized_correlation_matrix = features[numerical_features].corr()


plt.figure(figsize=(12, 10))
sns.heatmap(normalized_correlation_matrix, annot=True, cmap="coolwarm", linewidths=0.5)
plt.title("Correlation Matrix After Normalization")
plt.show()


In [ ]:

# Train a simple Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(features[numerical_features], targets['DEFAULT'])

# Plot feature importance
importances = pd.Series(rf_model.feature_importances_, index=numerical_features)
importances.sort_values().plot(kind='barh', figsize=(10, 6), title="Feature Importance (Random Forest)")
plt.show()


In [ ]:
correlation_with_target = processed_data.corr()['DEFAULT'].sort_values(ascending=False)
print("Correlation of Features with Target:")
print(correlation_with_target)
